1.使用单线程对文本分类的Naive Bayes模型的超参数组合执行网格搜索

In [27]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
news=fetch_20newsgroups(subset='all')
print len(news.data)
print news.data[0]

18846
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [28]:
from sklearn.cross_validation import train_test_split
# 选取前3000条新闻文本进行分割
X_train,X_test,y_train,y_test=train_test_split(news.data[:3000],news.target[:3000],test_size=0.25,random_state=33)

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

# 导入pipeline，使用pipeline简化系统搭建流程（简化代码），将文本抽取与分类器模型串联起来
from sklearn.pipeline import Pipeline
clf=Pipeline([('vect',TfidfVectorizer(stop_words='english',analyzer='word')),('svc',SVC())])

# 这里需要试验的2个超参数的个数分别是4、3，svc_gamma的参数共有10^-2,10^-1...，则共有12种超参数组合，12种不同参数下的模型
parameters={'svc_gamma':np.logspace(-2,1,4),'svc_C':np.logspace(-1,1,3)}

# 导入网络搜索模块GridSearchCV
from sklearn.grid_search import GridSearchCV
# 将12组参数组合、初始化的Pipeline和3折交叉验证的要求全部告诉GridSearchCV，注意refit=True的设定
gs=GridSearchCV(clf,parameters,verbose=2,refit=True,cv=3)

# 执行单线程网络搜索
%time_=gs.fit(X_train,y_train)
gs.best_params_,gs.best_score_

print gs.score(X_test,y_test)

UsageError: Line magic function `%time_` not found.


注：

- np.logspace(a,b,c):创建等比数列，a为起始点，b为终点，c为元素个数;np.linspace(a,b,c):创建等差数列
- refit=True: 使程序以交叉验证训练集得到的最佳超参数重新对所有可用的训练集合开发集进行，作为最终用于性能评估的最佳模型参数

结果分析：
- 使用单线程网格搜索技术对朴素贝叶斯模型在文本分类任务中的超参数组合进行调优，共有12组超参数X3折交叉验证=36项独立运行的计算任务。寻找到的最佳超参数组合在测试集上所能达成的最高分类准确性为82.27%。
- 缺点：耗时
- 优点：一旦获取到好超参数组合，则可以保持一段时间使用，是一劳永逸提高模型性能的方法

2.使用多线程对文本分类的Naive Bayes模型的超参数组合执行并行化的网络搜索

In [26]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
news=fetch_20newsgroups(subset='all')

from sklearn.cross_validation import train_test_split
# 选取前3000条新闻文本进行分割
X_train,X_test,y_train,y_test=train_test_split(news.data[:3000],news.target[:3000],test_size=0.25,random_state=33)

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

# 导入pipeline，使用pipeline简化系统搭建流程（简化代码），将文本抽取与分类器模型串联起来
from sklearn.pipeline import Pipeline
clf=Pipeline([('vect',TfidfVectorizer(stop_words='english',analyzer='word')),('svc',SVC())])

# 这里需要试验的2个超参数的个数分别是4、3，svc_gamma的参数共有10^-2,10^-1...，则共有12种超参数组合，12种不同参数下的模型
parameters={'svc_gamma':np.logspace(-2,1,4),'svc_C':np.logspace(-1,1,3)}

# 导入网络搜索模块GridSearchCV
from sklearn.grid_search import GridSearchCV

# 将12组参数组合、初始化的Pipeline和3折交叉验证的要求全部告诉GridSearchCV，注意refit=True的设定
# 初始化配置并行网络搜索，n_jobs=-1代表使用该计算机的全部CPU
gs=GridSearchCV(clf,parameters,verbose=2,refit=True,cv=3,n_jobs=-1)

# 执行多线程并行网络搜索
%time_=gs.fit(X_train,y_train)
gs.best_params_,gs.best_score_

# 输出最佳模型在测试集上的准确性
print gs.score(X_test,y_test)

UsageError: Line magic function `%time_` not found.


结果分析：

- 相较于单线程，多线程的计算时间仅为51.8秒，且准确性仍为82.27%
- 并行搜索：有效利用多核处理器的计算资源，几乎成倍提升运算速度，节省最佳超参数中护额的搜索时间